# Asignación y Desenmascarado

Saquemos las máscaras a los tweets

In [94]:
import pandas as pd

df_titulares = pd.read_csv("../data/titulos_filtrados.csv")

df_titulares = df_titulares.rename(columns={"Unnamed: 0": "id"}).set_index("id")

df_titulares

,medio,fecha,titulo
id,,,
23,la izquierda diario,2019-09-21 00:00:00,Barrio Aeroclub: la Izquierda propone urbaniza...
24,la izquierda diario,2019-09-21 00:00:00,Elecciones municipales: el #FITUnidad busca fo...
31,la izquierda diario,2019-09-21 00:00:00,[Videos] Mirá todos los spots del Frente de Iz...
65,la nación,2019-09-22 00:00:00,Tipo de cambio: por qué nos preocupa el dólar ...
75,la izquierda diario,2019-09-22 00:00:00,Brutalidad de la Policía de Urtubey en el reci...
...,...,...,...
22608,prensa obrera,2019-10-27 23:05:46,Declaración del Partido Obrero-Frente de Izqui...
22611,la nación,2019-10-27 23:16:00,"Resultados elecciones 2019: Alberto Fernández,..."
22612,la nación,2019-10-27 23:21:00,Elecciones 2019. La llamada de Macri a Alberto...


In [95]:
import pandas as pd
import glob
import numpy as np
import re



paths = glob.glob("../data/etiquetados_final/*.xlsx")

dfs = []
annotators = []

for path in paths:
    print(path)
    name = os.path.basename(path).split(".")[0]
    
    df = pd.read_excel(path, engine='openpyxl', index_col=0)
    df = df.iloc[3:]
    df = df.loc[
        df.index.notna()
    ]

    #df.index = df.index.astype(int)
    annotators.append(name)

    df.rename(columns={df.columns[0]: "id", "TITULO": "titulo_mask", " ": "etiquetado", "Etiquetado?": "etiquetado"}, inplace=True)

    # Por algún motivo perdimos este titular...
    ids = [x for x in df.index if x not in {39}]
    
    df.loc[ids, "titulo"] = df_titulares.loc[ids, "titulo"]
    df.loc[ids, "medio"] = df_titulares.loc[ids, "medio"]
    # KJJJJ qué desastre esto
    df.loc[39, "titulo"] = "Entre el milagro de Macri y la grieta del peronismo"
    df.loc[39, "medio"] = "la nación"
    df["anotador"] = name
    df.reset_index(inplace=True)
    dfs.append(df)
    
annotators = sorted(annotators)
annotator_df = pd.concat(dfs).set_index(["index", "anotador"])

../data/etiquetados_final/e3.xlsx
../data/etiquetados_final/et1.xlsx
../data/etiquetados_final/e2.xlsx


In [96]:

annotator_df

,,titulo_mask,etiquetado,TARGET A,TARGET B,TARGET C,TARGET D,TARGET E,TARGET F,titulo,medio
index,anotador,,,,,,,,,,
23,e3,Barrio Aeroclub: [TARGET D] propone urbaniza...,SI,NaN,NaN,NaN,POS,NaN,NaN,Barrio Aeroclub: la Izquierda propone urbaniza...,la izquierda diario
24,e3,Elecciones municipales: el #FITUnidad busca fo...,SI,NaN,NaN,NaN,POS,NaN,NaN,Elecciones municipales: el #FITUnidad busca fo...,la izquierda diario
31,e3,[Videos] Mirá todos los spots del [TARGET D] ...,SI,NaN,NaN,NaN,POS,NaN,NaN,[Videos] Mirá todos los spots del Frente de Iz...,la izquierda diario
39,e3,Entre el milagro de [TARGET C] y la grieta del...,SI,NaN,NaN,POS,NaN,NEG,NaN,Entre el milagro de Macri y la grieta del pero...,la nación
65,e3,Tipo de cambio: por qué nos preocupa el dólar ...,NO,NaN,NaN,NaN,NaN,NaN,NaN,Tipo de cambio: por qué nos preocupa el dólar ...,la nación
...,...,...,...,...,...,...,...,...,...,...,...
22608,e2,Declaración del Partido Obrero-[TARGET D] a...,SI,NaN,NaN,NaN,NEU,NaN,NaN,Declaración del Partido Obrero-Frente de Izqui...,prensa obrera
22611,e2,"Resultados elecciones 2019: [TARGET E] , tras ...",SI,NaN,NaN,NaN,NaN,POS,NaN,"Resultados elecciones 2019: Alberto Fernández,...",la nación
22612,e2,Elecciones 2019. La llamada de [TARGET C] a Al...,SI,NaN,NaN,POS,NaN,NaN,NaN,Elecciones 2019. La llamada de Macri a Alberto...,la nación


## Problemas de etiquetado

Si tiene algo que no sea POS, NEG, NEU => lo "blanqueo"

In [97]:
import numpy as np

label_cols = ['TARGET A', 'TARGET B', 'TARGET C', 'TARGET D',
       'TARGET E', 'TARGET F']

for label in label_cols:
    problematic = (annotator_df[label].notna() & ~annotator_df[label].apply(lambda x: x in {"POS", "NEG", "NEU"}))
    annotator_df.loc[problematic, label] = np.nan

## Asignación

Primero, veamos cómo asignarlo

In [98]:
ids = dfs[0]["index"]

rejected = []

accepted_ids = []

for _, row in dfs[0].iterrows():
    idx = row["index"]
    labels = annotator_df.loc[idx]

    assert len(labels) == 3
    annotated = labels[label_cols].notna()

    """
    Chequeamos si la anotación difirió
    """
    if not ( all(annotated.iloc[0] == annotated.iloc[1]) and all(annotated.iloc[1] == annotated.iloc[2])):
        """
        print("="*80 + "\n" * 2)
        print(idx, row["titulo"])
        print(annotated.sum(axis=1))
        print(annotated)
        """
        rejected.append(idx)
    else:
        accepted_ids.append(idx)

print("Titulares aceptados: ", len(accepted_ids))
print("Titulares descartados: ", len(rejected))

Titulares aceptados:  2205
Titulares descartados:  48


Ok, quedémonos con accepted_ids

Para cada titular, hagamos lo siguiente:

- Votación mayoritaria (elegimos al que tenga 2 votos)
- Si no ocurre...mmm, tiremos el titular mejor

In [100]:

annotator_df.loc[2010]

,titulo_mask,etiquetado,TARGET A,TARGET B,TARGET C,TARGET D,TARGET E,TARGET F,titulo,medio
anotador,,,,,,,,,,
e3,"Diez días antes de las elecciones, [TARGET C] ...",SI,NaN,NaN,POS,NaN,NaN,NaN,"Diez días antes de las elecciones, Mauricio Ma...",infobae
et1,"Diez días antes de las elecciones, [TARGET C] ...",SI,NaN,NaN,POS,NaN,NEG,NaN,"Diez días antes de las elecciones, Mauricio Ma...",infobae
e2,"Diez días antes de las elecciones, [TARGET C] ...",SI,NaN,NaN,POS,NaN,NEU,NaN,"Diez días antes de las elecciones, Mauricio Ma...",infobae


In [103]:
empty = [1601, 2163]
completed = [2010, 2049]

[x for x in rejected if x not in empty and x not in completed]


[2219,
 3296,
 3537,
 4253,
 4411,
 5677,
 6379,
 6562,
 6566,
 6571,
 6603,
 7438,
 8387,
 8392,
 8935,
 10205,
 11154,
 11229,
 11478,
 11688,
 11982,
 12082,
 12266,
 12749,
 13076,
 13900,
 14286,
 14378,
 15166,
 17468,
 18149,
 18198,
 18448,
 18474,
 18999,
 19022,
 21820,
 21883,
 22104,
 22222,
 22224,
 22312,
 22456,
 22583]

In [73]:
label = "TARGET C"



index  anotador
23     e3          False
24     e3          False
31     e3          False
39     e3          False
65     e3          False
                   ...  
22608  e2          False
22611  e2          False
22612  e2          False
22616  e2          False
22617  e2          False
Name: TARGET C, Length: 6759, dtype: bool

In [64]:
annotator_df[label_cols]["TARGET C"].str

In [4]:
import re
from tqdm.auto import tqdm

def get_targets(instance):
    mascarado = re.sub(r"\s+", " ", instance["titulo_mask"]).strip()
    titulo = instance["titulo"]

    i = 0
    j = 0

    found_targets = {}

    while i < len(titulo):
        # print("="*80)
        # print(titulo[i:])
        # print(mascarado[j:])
        # print(found_targets)
        if titulo[i] == mascarado[j]:
            i += 1
            j += 1
        else:
            "son distintos, tiene que ser un target"
            #import ipdb; ipdb.set_trace()
            assert mascarado[j] == "["
            
            next_idx = mascarado[j:].index("]") + j 
            
            target = mascarado[j+1:next_idx]

            j = next_idx + 1

            target_name = ""
            if j >= len(mascarado)-1:
                """
                Listo, me quedo con todo
                """
                target_name = titulo[i:]
                i = len(titulo)
            else:
                """
                Todavía queda algo
                """
                
                pattern = mascarado[j:j+2]
                next = titulo[i:].index(pattern)
                target_name = titulo[i:i+next]
                i = i + next 
            
            if target in found_targets:
                found_targets[target].append(target_name)
            else:
                found_targets[target] = [target_name]
    return found_targets



In [5]:


problematic = []

dataset = []

for idx in tqdm(accepted_ids):
    annotated_columns = [col for col, annotated in annotator_df.loc[idx, label_cols].notna().all().items() if annotated]

    # Agarro un etiquetador, total da igual
    row = annotator_df.loc[(idx, "et1")]

    ret = {k:row[k] for k in ["titulo", "titulo_mask"]}
    ret["id"] = idx
    mascara = re.sub(r"\s+", " ", ret["titulo_mask"]).strip()
    mascara = re.sub(r"(\])\s+([^\w\s“])", r"\1\2", mascara)

    ret["titulo_mask"] = mascara 
    
    ret["targets"] = {}
    """
    Asignamos
    """
    for col in annotated_columns:
        counts = annotator_df.loc[idx, col].value_counts()

        """
        Si no hay ningún máximo (es decir, si los votos son POS, NEG, NEU) => descartamos instancia
        """
        if counts.max() == 1:
            problematic.append(idx)
        else:
            label = counts.index[counts.argmax()]
            ret["targets"][col] = label


    if ret["targets"]:
        try:
            ret["mapping"] = get_targets(ret)
        except:
            pass
        dataset.append(ret)
print(f"Hay {len(problematic)} casos problemáticos de etiquetas (anotaron POS, NEG, NEU)")

100%|██████████| 2183/2183 [00:08<00:00, 244.53it/s]

Hay 114 casos problemáticos de etiquetas (anotaron POS, NEG, NEU)


In [6]:
pd.options.display.max_colwidth = 200

annotator_df.loc[problematic]["titulo_mask"].apply(lambda x: x.count("TARGET")).value_counts() / 3

1    80.0
2    24.0
3     4.0
0     1.0
Name: titulo_mask, dtype: float64

In [7]:
set_ids = []

def set_mapping(idx, mapping, keep=False):
    if keep:
        set_ids.append(idx)
    instance = next(x for x in dataset if x["id"] == idx)

    for k, v in mapping.items():
        if not isinstance(v, list):
            mapping[k] = [v]
    instance["mapping"] = mapping


set_mapping(21210, {"TARGET C": ["Juntos por el Cambio", "Macri"]})
set_mapping(22428, {"TARGET C": ["Juntos por el Cambio"]})
set_mapping(4806, {"TARGET D": ["Del Caño", "Del Plá", "FIT"]})

set_mapping(263, {"TARGET D": "Frente de Izquierda"})
set_mapping(639, {"TARGET E": "Alberto"})
set_mapping(690, {"TARGET D": ["Nicolás del Caño", "Romina del Pla", "FIT U"]})
set_mapping(827, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1042, {"TARGET C": ["Pichetto", "Macri"]})
set_mapping(1094, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1167, {"TARGET C": ["Juntos por el Cambio", "Macri"] })
set_mapping(1264, {"TARGET E": ["Alberto Fernández", "Cristina Kirchner"]})
set_mapping(1282, {"TARGET C": ["Miguel Ángel Pichetto", "Macri"], "TARGET E": ["Fernández", "Cristina"]})
set_mapping(1287, {"TARGET B": ["Roberto Lavagna", "Juan Manuel Urtubey"]})
set_mapping(1291, {"TARGET C": "Gobierno", "TARGET E": ["Alberto F.", "Cristina Kirchner"]}, keep=False)
set_mapping(2060, {"TARGET C": ["Gobierno", "Mauricio Macri"]}, keep=False)
set_mapping(2057, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(2158, {"TARGET C": ["Gobierno", "Macri"]}, keep=False)
set_mapping(2423, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"}, keep=False)
set_mapping(2434, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
set_mapping(2725, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
set_mapping(2835, {"TARGET C": ["Pichetto", "Gobierno"]})

set_mapping(3446, {"TARGET C": ["Gobierno", "Macri"]})
set_mapping(4388, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(4600, {"TARGET E": "Alberto"})
set_mapping(4783, {"TARGET A": "Espert", "TARGET D": "Nico del Caño"})
set_mapping(4885, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(4936, {"TARGET B": ["Roberto Lavagna", "Consenso Federal"], "TARGET C": "Gobierno"})
set_mapping(5061, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(5338, {"TARGET C": "Mauricio Macri", "TARGET E": "peronismo"})
set_mapping(5453, {"TARGET E": "Alberto"})
set_mapping(6301, {"TARGET C": "Juntos por el Cambio"})
set_mapping(5070, {"TARGET A": "José Luis Espert", "TARGET D": "Nicolás del Caño"})
set_mapping(5804, {"TARGET E": ["Alberto Fernández", "gobierno k"], "TARGET C": ["este gobierno"]})
set_mapping(6014, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(6605, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7429, {"TARGET C": "Gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(7725, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(8131, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(10204, {"TARGET E": "PJ"})
set_mapping(10513, {"TARGET B": "Roberto Lavagna", "TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(10748, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(11474, {"TARGET C": "gobierno", "TARGET E": "Alberto Fernández"})
set_mapping(11528, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(12462, {"TARGET C": "Gobierno", "TARGET E": "Cristina"})
set_mapping(13068, {"TARGET D": "Nicolás del Caño"})
set_mapping(14842, {"TARGET C": ["Juntos por el Cambio", "Macri"]})
set_mapping(14855, {"TARGET C": ["Gobierno", "Macri"]})

set_mapping(14150, {"TARGET C": "Macri", "TARGET E": "Alberto F."})
set_mapping(15856, {"TARGET B": "Roberto Lavagna", "TARGET C": "Gobierno"})
set_mapping(16255, {"TARGET C": ["Gobierno", "Macri"]})
set_mapping(16134, {"TARGET B": "Lavagna", "TARGET C": "Gobierno"})
set_mapping(16424, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(17935, {"TARGET C": "Gobierno", "TARGET E": "CFK"})
set_mapping(18449, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(18231, {"TARGET D": "la izquierda"})
set_mapping(18467, {"TARGET A": "José Luis Espert", "TARGET C": "Mauricio Macri", "TARGET E": "kirchnerismo"})
set_mapping(19447, {"TARGET C": "Gobierno"})
set_mapping(20433, {"TARGET C": "Mauricio Macri"})
set_mapping(21233, {"TARGET C": "Mauricio Macri", "TARGET E": "Alberto Fernández"})
set_mapping(21483, {"TARGET C": "Gobierno", "TARGET E": "peronismo"})
set_mapping(22381, {"TARGET E": "Alberto Fernández"})
#set_mapping(2434, {"TARGET C": ["Miguel Ángel Pichetto", "Gobierno"]})
delete_ids = [2219]

dataset = [x for x in dataset if x["id"] not in delete_ids]

[x for x in dataset if "mapping" not in x or x["id"] in set_ids]

[]

In [8]:
ignore = {639}

for instance in dataset:
    if instance["id"] in ignore:
        continue
    if instance["titulo_mask"].count("TARGET") != sum([1 if isinstance(v, str) else len(v) for k, v in instance["mapping"].items()]):
        print("="*80)
        print(instance["id"])
        print(instance["titulo"])
        print(instance["titulo_mask"], instance["targets"], instance["mapping"])

10204
El PJ quiere revisar tarifas y asociarlas con el costo de producción
El PJ quiere revisar tarifas y asociarlas con el costo de producción {'TARGET E': 'NEU'} {'TARGET E': ['PJ']}


In [9]:
def clean_instance(instance):
    ret = {k:instance[k] for k in ["titulo", "titulo_mask", "id"]}

    ret["labels"] = {}
    
    for k, label in instance["targets"].items():
        names = instance["mapping"][k]

        if isinstance(names, str):
            names = [names]

        for name in names:
            ret["labels"][name] = label
    return ret

clean_dataset = []
for instance in dataset:
    clean_dataset.append(clean_instance(instance))
#clean_dataset = [clean_instance(instance) for instance in dataset]
    
#clean_dataset = [clean_instance(instance) for instance in dataset]
print("Cantidad de titulares: ", len(clean_dataset))
print("Cantidad de targets: ", sum([len(l["labels"])for l in clean_dataset]))



Cantidad de titulares:  1930
Cantidad de targets:  2374


Saquemos espacios espurios de 

## Sanity check

In [10]:
ignore_ids = {690, 2158}
for instance in clean_dataset:
    if instance["id"] in ignore_ids:
        continue
    for k, v in instance["labels"].items():
        if k.lower() not in instance["titulo"].lower():
            print(instance)
            print(k)

{'titulo': 'Por error, y en plena veda, TN dio por ganador a Alberto Férnandez', 'titulo_mask': 'Por error, y en plena veda, TN dio por ganador a [TARGET E] érnandez', 'id': 22381, 'labels': {'Alberto Fernández': 'NEU'}}
Alberto Fernández


In [11]:
annotator_df.loc[[5070]]

titulo_mask  \
id   anotador                                                             
5070 e2        [TARGET A]  y una ironía sobre el libro de [TARGET D]      
     e3        [TARGET A]  y una ironía sobre el libro de [TARGET D]      
     et1       [TARGET A]  y una ironía sobre el libro de [TARGET D]      

              etiquetado TARGET A TARGET B TARGET C TARGET D TARGET E  \
id   anotador                                                           
5070 e2               SI      NEG      NaN      NaN      NEG      NaN   
     e3               SI      POS      NaN      NaN      NEG      NaN   
     et1              SI      NEU      NaN      NaN      NEU      NaN   

              TARGET F  \
id   anotador            
5070 e2            NaN   
     e3            NaN   
     et1           NaN   

                                                                         titulo  
id   anotador                                                                    
5070 e2        José Luis Espert y una ironía sobre el libro de Nicolás del Caño  
     e3        José Luis Espert y una ironía sobre el libro de Nicolás del Caño  
     et1       José Luis Espert y una ironía sobre el libro de Nicolás del Caño

In [12]:
for instance in clean_dataset:

    if instance["id"] in {639, 2057, 4783, 5070, 5338, 10204}:
        continue
    
    if instance["titulo_mask"].count("TARGET") != len(instance["labels"]):
        print(instance)

{'titulo': 'El dilema de Macri: ¿cruzar el Rubicón y emitir o dejarle el balurdo a Alberto F.?', 'titulo_mask': 'El dilema de [TARGET C]: ¿cruzar el Rubicón y emitir o dejarle el balurdo a [TARGET E].?', 'id': 10295, 'labels': {'Macri': 'NEG'}}
{'titulo': 'Mauricio Macri y Roberto Lavagna rechazan la suba de Bienes Personales que planteó Alberto Fernández', 'titulo_mask': '[TARGET C] y [TARGET B] rechazan la suba de Bienes Personales que planteó [TARGET E]', 'id': 10513, 'labels': {'Alberto Fernández': 'NEG'}}
{'titulo': 'Cristina juega al silencio pero influye en la campaña y los proyectos de gobierno', 'titulo_mask': '[TARGET E] juega al silencio pero influye en la campaña y los proyectos de GOBIERNO ([TARGET C])', 'id': 12462, 'labels': {'Gobierno': 'NEU'}}
{'titulo': 'Macri vs. Macri: cómo cambió el discurso del Presidente desde el histórico cruce con Scioli', 'titulo_mask': '[TARGET C] vs. [TARGET C]: cómo cambió el discurso del Presidente desde el histórico cruce con Scioli', 'id

Nada grave

In [13]:
clean_dataset = [{k:v for k, v in instance.items() if k != "titulo_mask"} for instance in clean_dataset]

len(clean_dataset)

1930

In [14]:
import json

with open("../data/dataset.json", "w+") as f:
    json.dump(clean_dataset, f, indent=4)